# LIBRARIES AND REQUIREMENTS

In [1]:
import zipfile
import os
from google.colab import drive

if os.path.exists('Detic'):
  print("Drive Mounted")
else:
  drive.mount('/content/drive/', force_remount=True)
  !unzip -qq /content/drive/MyDrive/INTERPRETABILITY/Zip\ Files/Detic.zip -d /
  !unzip -qq /content/drive/MyDrive/INTERPRETABILITY/Zip\ Files/detectron2.zip -d /
  !unzip -qq /content/drive/MyDrive/INTERPRETABILITY/Zip\ Files/Interpretability.zip -d /content/
  !unzip -qq /content/drive/MyDrive/INTERPRETABILITY/Zip\ Files/in_painting.zip -d /content



Mounted at /content/drive/


In [2]:
import sys
sys.path.append("/content/Interpretability")
outer_path = "/content/Interpretability/"

# if os.path.exists('/content/classes'):
#   print("Unzipped dataset")
# else:
#   !unzip -qq /content/Interpretability/datasets/LigiLog-100_coco_classes.zip -d /content/



In [3]:
# Install detectron2
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
# print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

!pip install click==7.1.2

!pip install -q  -e detectron2

# clone and install Detic
!pip install -q  -r Detic/requirements.txt
!pip install -q -r Detic/third_party/CenterNet2/requirements.txt




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 151 kB 72.1 MB/s 
     |████████████████████████████████| 1.4 MB 64.9 MB/s 
     |████████████████████████████████| 509 kB 73.0 MB/s 
     |████████████████████████████████| 248 kB 76.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 96 kB 7.2 MB/s 
     |████████████████████████████████| 843 kB 60.0 MB/s 
     |████████████████████████████████| 117 kB 71.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but

In [4]:
# Some basic setup:
# Setup detectron2 logger
import sys
outer_path = "/content/detectron2/"
sys.path.append(outer_path)
from PIL import Image
import detectron2 
from detectron2.utils.logger import setup_logger
setup_logger()
from google.colab.patches import cv2_imshow
from collections import defaultdict


from IPython.display import clear_output
# import some common libraries
import sys
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from scipy.ndimage.filters import gaussian_filter
import importlib
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
np.set_printoptions(threshold=sys.maxsize)
# Detic libraries
sys.path.insert(0, '/content/Detic/third_party/CenterNet2')
from centernet.config import add_centernet_config
from Detic.detic.config import add_detic_config
from Detic.detic.modeling.utils import reset_cls_test
import matplotlib.pyplot as plt
import pandas as pd 
import os
import cv2

try:
  import models as md
except:
  !pip install timm
  !pip install torchsort
  import models as md
from tqdm import tqdm
from glob import glob
import torch as tr
import torch.nn as nn
from PIL import Image
from torch.nn.functional import interpolate
import torchvision.transforms as T
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from scipy.stats import percentileofscore
import numpy as np
from torch.utils import data
from google.colab.patches import cv2_imshow
from scipy.ndimage.filters import gaussian_filter


import matplotlib.pyplot as plt
from torch.autograd import Variable
device = tr.device("cuda:0" if tr.cuda.is_available() else "cpu")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchsort: filename=torchsort-0.1.9-cp37-cp37m-linux_x86_64.whl size=2768795 sha256=22b8f42f9b3b59b83f49454080096f910f063aa850a837b6875da89fdbeb3727
  Stored in directory: /root/.cache/pip/wheels/78/20/69/cc894d3d65cb58f99ada9462cd8fb76e3beaac185efa06df3e
Successfully built torchsort


In [5]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
import clip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2s9498ad
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-2s9498ad


# OBJECT SEGMENTATION

In [6]:

def get_predictor():
	# Build the detector and download our pretrained weights
	cfg = get_cfg()
	add_centernet_config(cfg)
	add_detic_config(cfg)
	cfg.merge_from_file("/content/Detic/configs/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.yaml")
	cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/detic/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth'
	cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
	cfg.MODEL.ROI_BOX_HEAD.ZEROSHOT_WEIGHT_PATH = 'rand'
	cfg.MODEL.ROI_HEADS.ONE_CLASS_PER_PROPOSAL = True # For better visualization purpose. Set to False for all classes.
	# cfg.MODEL.DEVICE='cpu' # uncomment this to use cpu-only mode.
	predictor = DefaultPredictor(cfg)

	BUILDIN_CLASSIFIER = {
	    'lvis': 'Detic/datasets/metadata/lvis_v1_clip_a+cname.npy',
	    'objects365': 'Detic/datasets/metadata/o365_clip_a+cnamefix.npy',
	    'openimages': 'Detic/datasets/metadata/oid_clip_a+cname.npy',
	    'coco': 'Detic/datasets/metadata/coco_clip_a+cname.npy',
	}

	BUILDIN_METADATA_PATH = {
	    'lvis': 'lvis_v1_val',
	    'objects365': 'objects365_v2_val',
	    'openimages': 'oid_val_expanded',
	    'coco': 'coco_2017_val',
	}

	vocabulary = 'lvis' # change to 'lvis', 'objects365', 'openimages', or 'coco'
	metadata = MetadataCatalog.get(BUILDIN_METADATA_PATH[vocabulary])
	classifier = BUILDIN_CLASSIFIER[vocabulary]
	num_classes = len(metadata.thing_classes)
	reset_cls_test(predictor.model, classifier, num_classes)
	return predictor, metadata


predictor, metadata = get_predictor()


def segmentation(im_name, input_type="img"):
	
	if input_type!="img":
		im = cv2.imread(im_name)
		im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
	 
	outputs = predictor(im)
	classes = outputs["instances"].to("cpu").pred_classes.tolist()
	class_names = [metadata.thing_classes[cl] for cl in classes]
	return outputs, class_names



/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth: 702MB [00:35, 19.6MB/s]                           


[09/13 14:36:18 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                          | Names in Checkpoint                                                                                                                         | Shapes                                                   |
|:--------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------|
| backbone.bottom_up.layers.0.blocks.0.attn.*             | backbone.bottom_up.layers.0.blocks.0.attn.{proj.bias,proj.weight,qkv.bias,qkv.weight,relative_position_bias_table,relative_position_index}  | (128,) (128,128) (384,) (384,128) (169,4) (49,49)        |
| backbone.bottom_up.layers.0.blocks.0.mlp.fc1.*          | backbone.bottom_up.layers.0.blocks.0.mlp.fc1.{bias,weight}            

# MEMORABILITY MODEL

In [7]:
model = md.EfficientNet()

model.cuda()
model.eval()

name_of_model = "baseline_2021-12-20"
if os.path.isfile(outer_path+"parameters/"+name_of_model+".pmt"):
    # print("Loading parameters from file")
    model.load_state_dict(tr.load(outer_path+"parameters/"+name_of_model+".pmt"))

model.to(device);


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b5_ns-6f26d0cf.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b5_ns-6f26d0cf.pth


# IMAGE TRANSFORMATIONS

In [8]:
def ImageTransform(path, ip_type="filename"):
  if ip_type=="filename":
    file_name = path
    img = Image.open(file_name)
    img = img.convert("RGB")

  elif ip_type=="otr":
    img = Image.fromarray(path)
    
  else:
    img = path

  transform1 = T.Compose([T.ToTensor()])

  x = transform1(img)*255

  img_mean = tr.Tensor([123.675, 116.28, 103.53]).view(3, 1, 1)
  img_std = tr.Tensor([58.395, 57.12, 57.375]).view(3, 1, 1)

  x = x.to(dtype=tr.float)

  x = interpolate(
      x.unsqueeze(0),
      size=(456,456),#(model.img_size, model.img_size),
      mode='bilinear',
      align_corners=False,
      recompute_scale_factor=False
  ).squeeze(0)

  x = ((x - img_mean) / img_std)#.unsqueeze(0)
  return np.array(img), x


In [9]:
def alter_mask(mask, val = 50):
  # mask = Image.fromarray(mask)
  # mask = mask[:, :, ::-1].copy() 
  plt.imsave("temp_mask.png", mask)
  mask = cv2.imread("temp_mask.png", cv2.THRESH_BINARY)
  inp_mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (val, val)))
  inp_mask = cv2.dilate(inp_mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (val, val)))
  return inp_mask


# IMAGE-BLENDING

In [10]:
def image_blend(background, img, mask, x, y):
    '''
    Arguments:
    background - background image in CV2 RGB format
    img - image of object in CV2 RGB format
    mask - mask of object in CV2 RGB format
    x, y - coordinates of the center of the object image
    0 < x < width of background
    0 < y < height of background
    
    Function returns background with added object in CV2 RGB format
    
    CV2 RGB format is a numpy array with dimensions width x height x 3
    '''
    # print(background.shape, type(background))
    # print(img.shape, type(img))
    bg = background.squeeze(0).permute(1,2,0).detach().numpy()
    
    img = img.squeeze(0).permute(1,2,0).detach().numpy()
    mask = np.invert(mask)
    # bg = background.clone()
    # bg = np.expand_dims(bg, 2)
    # print(bg.shape, img.shape, mask.shape, x, y)
    h_bg, w_bg = bg.shape[0], bg.shape[1]
    h, w = img.shape[0], img.shape[1]
    
    x = x - int(w/2)
    y = y - int(h/2)    
    
    # mask_boolean = mask[:,:,0] == 0
    mask_boolean = mask[:,:] == 0
    mask_rgb_boolean = np.stack([mask_boolean, mask_boolean, mask_boolean], axis=2)
    
    if x >= 0 and y >= 0:
        h_part = h - max(0, y+h-h_bg) 
        w_part = w - max(0, x+w-w_bg) 
        bg[y:y+h_part, x:x+w_part, :] = bg[y:y+h_part, x:x+w_part, :] * ~mask_rgb_boolean[0:h_part, 0:w_part, :] + (img * mask_rgb_boolean)[0:h_part, 0:w_part, :]
        
    elif x < 0 and y < 0:
        h_part = h + y
        w_part = w + x
        bg[0:0+h_part, 0:0+w_part, :] = bg[0:0+h_part, 0:0+w_part, :] * ~mask_rgb_boolean[h-h_part:h, w-w_part:w, :] + (img * mask_rgb_boolean)[h-h_part:h, w-w_part:w, :]
        
    elif x < 0 and y >= 0:
        h_part = h - max(0, y+h-h_bg)
        w_part = w + x
        bg[y:y+h_part, 0:0+w_part, :] = bg[y:y+h_part, 0:0+w_part, :] * ~mask_rgb_boolean[0:h_part, w-w_part:w, :] + (img * mask_rgb_boolean)[0:h_part, w-w_part:w, :]
        
    elif x >= 0 and y < 0:
        h_part = h + y
        w_part = w - max(0, x+w-w_bg)
        bg[0:0+h_part, x:x+w_part, :] = bg[0:0+h_part, x:x+w_part, :] * ~mask_rgb_boolean[h-h_part:h, 0:w_part, :] + (img * mask_rgb_boolean)[h-h_part:h, 0:w_part, :]
    
    return bg


# IN-PAINTING MODEL

In [11]:
net = importlib.import_module("in_painting.src.model.aotgan")
inpaint_model = net.InpaintGenerator([1,2,5,8], 8)
inpaint_model.load_state_dict(torch.load("/content/in_painting/experiments/G0000000.pt", map_location='cpu'))



def postprocess(image):
    image = torch.clamp(image, -1., 1.)
    image = (image + 1) / 2.0 * 255.0
    image = image.permute(1, 2, 0)
    image = image.cpu().numpy().astype(np.uint8)
    return image



def inpaint(model, img, mask):
    mask1 = T.ToTensor()(mask.astype(int)).unsqueeze(0)

    image = T.ToTensor()(img)
    image = (image * 2.0 - 1.0).unsqueeze(0)

    image_masked = (image * (1 - mask1).float()) + mask1
    model.eval()

    with torch.no_grad():
      pred_img = model(image_masked, mask1 )
    
    # Using altered mask after altered prediction
    try:
      comp_imgs = (1 - mask1) * image + mask1 * pred_img
    except:
      w,h = mask1.shape[2:]

      pred_img = T.Resize((w,h))(pred_img)
      comp_imgs = (1 - mask1) * image + mask1 * pred_img
    comp_np = postprocess(comp_imgs[0])
    return comp_np



# MASK VALIDITY

In [12]:
def check_mask_validity(mask):
  w,h = mask.shape[0], mask.shape[1]

  total_size = w*h
  min_size = total_size*0.05
  max_size = total_size*0.8

  mask_size = mask[mask==255].shape[0]

  if mask_size>=min_size and mask_size<=max_size:
    return True
    
  else:
    return False



# CLIP EMBEDDING

In [13]:
# Get the clip_model
clip_model, _ = clip.load("ViT-B/32", device=device, jit=False)

# Function to generate the clip embeddings
def get_clip_embedding(img, input_type="otr"):

  if input_type=="org":
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,224))
    image = T.ToTensor()(image.astype(int)).unsqueeze(0).to(device)
  
  else:
    image = T.Resize(224)(img)
  

  with tr.no_grad():
    image_features = clip_model.encode_image(image)

  image_features = image_features.detach().cpu().numpy().flatten()

  return image_features


100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 51.7MiB/s]


# INFERENCE TEXT

In [14]:
def inference_save(original_prediction, inpaint_prediction, inpaint_displace_prediction, original_clip_embedding, clip_embedding, image_name, class_name, dpos):
  global compiled_result_list, inpaint_count, inpaint_disp_count
  save_dict = {}

  save_dict["Image_ID"] = image_name
  try:
    cn = "_".join([word.capitalize() for word in class_name.lower().split("_")])
  except:
    cn = class_name
  save_dict["Class_Name"] = cn

  # Save the original memorability scores
  for i in range(5):
    key_name = f'Original_Prediction_Memorability_Score_{i}'
    save_dict[key_name] = original_prediction[i]
  
  save_dict["Original_Prediction_Clip_Embedding"] = original_clip_embedding

  # Save the inpaint memorability scores
  for i in range(5):
    key_name = f'Inpaint_Prediction_Memorability_Score_{i}'
    save_dict[key_name] = inpaint_prediction[i]

  save_dict["Inpaint_Prediction_Clip_Embedding"] = clip_embedding[0]

  # Save the inpaint+displacement memorability scores
  for count, pos_name in enumerate(dpos.keys()):

    for i in range(5):
      key_name = f'Inpaint_w_Displacement_{pos_name}_Prediction_Memorability_Score_{i}'
      save_dict[key_name] = inpaint_displace_prediction[pos_name][i]

    key_name = f"Inpaint_w_Displacement_{pos_name}_Clip_Embedding"
    save_dict[key_name] = clip_embedding[count+1]

  # Append the current dictionary to the global list
  compiled_result_list.append(save_dict)

  # Save the original memorability heatmaps as images
  %cd Memorability_Heatmaps_Results
  !mkdir Original_Results
  for mem_plot in range(14):
    img = original_prediction[mem_plot+5]
    plt.imsave(f"/content/Memorability_Heatmaps_Results/Original_Results/Map_{mem_plot}.png", img, dpi=500)

  # Save the inpaint memorability heatmaps of the particular object mask as images
  !mkdir Inpaint_Results_"$inpaint_count"
  for mem_plot in range(14):
    img = inpaint_prediction[mem_plot+5]
    try:
      plt.imsave(f"/content/Memorability_Heatmaps_Results/Inpaint_Results_{inpaint_count}/Map_{mem_plot}.png", img, dpi=500)
    except:
      !mkdir -p Memorability_Heatmaps_Results/Inpaint_Results_"$inpaint_count"
      plt.imsave(f"/content/Memorability_Heatmaps_Results/Inpaint_Results_{inpaint_count}/Map_{mem_plot}.png", img, dpi=500)

  inpaint_count+=1

  # Save the inpaint + displacement memorability heatmaps of the particular object mask as images
  !mkdir Inpaint_w_Displacement_Results_"$inpaint_disp_count"

  for count, pos_name in enumerate(dpos.keys()):
    !mkdir Inpaint_w_Displacement_Results_"$inpaint_disp_count"/"$pos_name"
    for mem_plot in range(14):
      img = inpaint_displace_prediction[pos_name][mem_plot+5]
      try:
        plt.imsave(f"/content/Memorability_Heatmaps_Results/Inpaint_w_Displacement_Results_{inpaint_disp_count}/{pos_name}/Map_{mem_plot}.png", img, dpi=500)
      except:
        !mkdir -p Memorability_Heatmaps_Results/Inpaint_w_Displacement_Results_"$cn"/"$pos_name"
        plt.imsave(f"/content/Memorability_Heatmaps_Results/Inpaint_w_Displacement_Results_{inpaint_disp_count}/{pos_name}/Map_{mem_plot}.png", img, dpi=500)
  
  inpaint_disp_count+=1
  %cd ..

# MAIN FUNCTION

In [15]:
# GENERAL SETTINGS

# Set this to get results of a particular memorability score
mem_score_type = 0

# Set to True if object is to be removed only
# Set to False if object is to be removed and then inserted at a different position
remove = False

# Set to True to alter mask i.e. open and dilated it
alt_mask = True

# # If the above is set to False, mention the position of the object to be placed in the image
# # This is the position of the center of the object on the backgroung image
# x = 200
# y = 500

# Set this value to open and dilate the mask 
open = 50

# Save all predictions
original_prediction = []
# inpaint_prediction = defaultdict(list)
# inpaint_displace_prediction = defaultdict(list)

# List to store all results and save as csv
compiled_result_list = []

# Create a new folder to save the results
!mkdir -p Memorability_Heatmaps_Results/Original_Results

# Global variable to create folders for every mask
inpaint_count = 0

# Global variable to create folders for every mask
inpaint_disp_count = 0


In [ ]:
def main(img_name, img_path="/content/"):
  global compiled_result_list
  # Input image
  image_path = img_path
  img = cv2.imread(f'{image_path}{img_name}')
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  # Dictionary to specify position 
  # w,h = img.shape[0], img.shape[1]
  w, h = 456, 456
  dpos = {}
  dpos["Top_Left"] = [int(w//4), int(h*0.75)]
  dpos["Top_Right"] = [int(w*0.75), int(h*0.75)]
  dpos["Bottom_Left"] = [int(w//4), int(h//4)]
  dpos["Bottom_Right"] = [int(w*0.75), int(h//4)]
  dpos["Center"] = [int(w//2), int(h//2)]

  # Transform image to send to the memorability model 
  # _, model_input = ImageTransform(img, "image")

  model_input = T.ToTensor()(img).unsqueeze(0)

  # Call to object segmentation function
  outputs, class_names = segmentation(f'{image_path}{img_name}', input_type="org")

  # Get original clip embeddings
  original_clip_embedding = get_clip_embedding(img_name, "org")

  # print(model_input.squeeze(0).permute(1,2,0).detach().cpu().numpy().shape)
  plt.imsave("model_input.png", model_input.squeeze(0).permute(1,2,0).detach().cpu().numpy())
  _,mi = ImageTransform("model_input.png")
  # print(mi.shape)
  model_output = model(mi.unsqueeze(0).cuda())

  # Get output of the original model for all memorability scores
  for mem_score_type in range(5):
    original_prediction.append(model_output[0][0][mem_score_type].detach().cpu().numpy())

  # Get output of the original model for all heatmap
  for mem_plot in range(14):
    original_prediction.append(model_output[1].detach().cpu().squeeze(0).permute(2,1,0).numpy()[:,:,mem_plot])

  # Loop over all the masks
  for idx, name in enumerate(class_names):
    inpaint_prediction = []
    inpaint_displace_prediction = defaultdict(list)

    clip_embedding = []

    mask = outputs["instances"].to("cpu").pred_masks[idx,:,:].numpy()
    # mask_valid = check_mask_validity(mask)

    # if mask_valid==False:
    #   continue

    # Alter the mask
    # alt_mask = alter_mask(cv2.fromarray(mask), open)
    alt_mask = alter_mask(mask, open)

    # # INPAINT

    # Inpaint the image with the altered mask
    new_model_input = inpaint(inpaint_model, img, alt_mask)
    new_model_input = T.ToTensor()(new_model_input.astype(int)).unsqueeze(0)

    # Get the new model predictions
    plt.imsave("model_input.png", model_input.squeeze(0).permute(1,2,0).detach().cpu().numpy())
    _,mi = ImageTransform("model_input.png")
    model_output = model(mi.unsqueeze(0).cuda())

    key = f"mask_{idx}"
    for mem_score_type in range(5):
      # inpaint_prediction[key].append(model_output[0][:,mem_score_type])
      # print(model_output[0][0][mem_score_type].detach().cpu().numpy())
      inpaint_prediction.append(model_output[0][0][mem_score_type].detach().cpu().numpy())

    for mem_plot in range(14):
      # inpaint_prediction[key].append(model_output[1].detach().cpu().squeeze(0).permute(2,1,0).numpy()[:,:,mem_plot])
      inpaint_prediction.append(model_output[1].detach().cpu().squeeze(0).permute(2,1,0).numpy()[:,:,mem_plot])

    clip_embedding.append(get_clip_embedding(new_model_input.cuda()))

    # # INPAINT + DISPLACEMENT

    # Place object in specified position 
    for pos_name, pos in dpos.items():
      # print(pos)
      new_model_input = image_blend(new_model_input, model_input, mask, pos[0], pos[1])
    
      new_model_input = T.ToTensor()(new_model_input.astype(int)).unsqueeze(0)
      plt.imsave("model_input.png", model_input.squeeze(0).permute(1,2,0).detach().cpu().numpy())
      _,mi = ImageTransform("model_input.png")
      model_output = model(mi.unsqueeze(0).cuda())

      key = f"mask_{idx}_{pos_name}"
      
      for mem_score_type in range(5):
        inpaint_displace_prediction[pos_name].append(model_output[0][0][mem_score_type].detach().cpu().numpy())

      for mem_plot in range(14):
        inpaint_displace_prediction[pos_name].append(model_output[1].detach().cpu().squeeze(0).permute(2,1,0).numpy()[:,:,mem_plot])

      clip_embedding.append(get_clip_embedding(new_model_input.cuda()))

    # Call to the inference script to update the results list with values for the current mask
    inference_save(original_prediction, inpaint_prediction, inpaint_displace_prediction, original_clip_embedding, clip_embedding, img_name, name, dpos)

  
  # Save the tabular results to a csv file
  df = pd.DataFrame.from_dict(compiled_result_list) 
  df.to_csv (r'Inference.csv', index = False, header=True)
  !zip -r "/content/Memorability_Heatmaps_Results.zip" "/content/Memorability_Heatmaps_Results"

img_name = "input.jpg"
main(img_name)

